import and pip necessary libraries

In [2]:
import pandas as pd 
import time
import matplotlib.pyplot as plt
import numpy as np

! pip install scikit-surprise
from surprise import Dataset, Reader
from surprise import KNNBasic
from surprise import KNNBaseline
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise import SVD, SVDpp
from surprise.model_selection import cross_validate
from surprise.model_selection import KFold

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [3]:
filtered_data = pd.read_csv("filtered_data.csv")
filtered_data.head() 

,userId,productId,rating,timestamp
0,AO94DHGC771SJ,0528881469,5.0,1370131200
1,AMO214LNFCEI4,0528881469,1.0,1290643200
2,A3N7T0DY83Y4IG,0528881469,3.0,1283990400
3,A1H8PY3QHMQQA0,0528881469,2.0,1290556800
4,A24EV6RXELQZ63,0528881469,1.0,1317254400


In [4]:
# Display dataset information and basic statistics
print(filtered_data.info())
print(filtered_data.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1828344 entries, 0 to 1828343
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     object 
 1   productId  object 
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 55.8+ MB
None
             rating     timestamp
count  1.828344e+06  1.828344e+06
mean   4.209477e+00  1.340618e+09
std    1.200339e+00  6.384993e+07
min    1.000000e+00  9.292320e+08
25%    4.000000e+00  1.318464e+09
50%    5.000000e+00  1.361059e+09
75%    5.000000e+00  1.385251e+09
max    5.000000e+00  1.406074e+09


Data Preprocess

Item-Item Collaborative Filtering model

Transfer data into Surprise format, and split train and test data

In [5]:
reader = Reader(rating_scale=(1, 5))

surprise_data = Dataset.load_from_df(filtered_data[['userId', 'productId', 'rating']], reader)

SVD and SVD++

In [5]:
# Record the start time
start_time = time.time()

# Define the parameter grid for GridSearchCV
param_grid = {
    'n_epochs': [5, 10, 15],
    'lr_all': [0.002, 0.005, 0.008],
    'reg_all': [0.2, 0.4, 0.6]
}

# Perform GridSearchCV for SVD
grid_search_svd = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=5, n_jobs=-1)
grid_search_svd.fit(surprise_data)

# Print the best parameters for SVD
print("Best parameters for SVD: ", grid_search_svd.best_params['rmse'])

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


Best parameters for SVD:  {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.2}
Elapsed time: 1167.66 seconds


In [6]:
# Record the start time
start_time = time.time()

# Perform GridSearchCV for SVD++
grid_search_svdpp = GridSearchCV(SVDpp, param_grid, measures=['rmse', 'mae'], cv=5, n_jobs=-1)
grid_search_svdpp.fit(surprise_data)

# Print the best parameters for SVD++
print("Best parameters for SVD++: ", grid_search_svdpp.best_params['rmse'])

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Best parameters for SVD++:  {'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.2}
Elapsed time: 1260.62 seconds


In [7]:
# Record the start time
start_time = time.time()

# Cross-validate the best SVD model
best_svd = SVD(**grid_search_svd.best_params['rmse'])
cv_results_svd = cross_validate(best_svd, surprise_data, measures=['rmse', 'mae'], cv=5, verbose=True)

# Cross-validate the best SVD++ model
best_svdpp = SVDpp(**grid_search_svdpp.best_params['rmse'])
cv_results_svdpp = cross_validate(best_svdpp, surprise_data, measures=['rmse', 'mae'], cv=5, verbose=True)

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1227  1.1191  1.1180  1.1199  1.1178  1.1195  0.0018  
MAE (testset)     0.8529  0.8508  0.8495  0.8518  0.8504  0.8511  0.0012  
Fit time          40.21   41.25   41.19   41.35   41.19   41.04   0.42    
Test time         4.02    4.49    4.42    4.06    3.92    4.18    0.23    
Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1190  1.1199  1.1155  1.1167  1.1171  1.1176  0.0016  
MAE (testset)     0.8504  0.8508  0.8482  0.8492  0.8487  0.8495  0.0010  
Fit time          143.26  141.25  143.91  141.45  142.18  142.41  1.03    
Test time         10.02   9.13    9.32    9.23    9.30    9.40    0.32    
Elapsed time: 1037.98 seconds


Basic KNN

In [8]:
# Record the start time
start_time = time.time()

# Train the basic KNN model
basic_knn = KNNBasic(k=10, sim_options={'name': 'cosine', 'user_based': False})

# Run 5-fold cross-validation and print results.
cross_validate(basic_knn, surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2821  1.2831  1.2826  1.2809  1.2772  1.2812  0.0021  
MAE (testset)     0.8937  0.8939  0.8931  0.8924  0.8902  0.8927  0.0013  
Fit time          133.22  136.08  143.77  136.62  139.68  137.87  3.59    
Test time         8.74    9.01    8.81    9.88    10.10   9.30    0.57    
Elapsed time: 762.00 seconds


KNN Baseline

In [6]:
# Record the start time
start_time = time.time()

# Train the best k-NN model using the best parameters
base_knn = KNNBaseline(k=10, sim_options={'name': 'cosine', 'user_based': False})

# Run 5-fold cross-validation and print results.
cross_validate(base_knn, surprise_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time:.2f} seconds")

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2025  1.1987  1.1990  1.1997  1.2009  1.2002  0.0014  
MAE (testset)     0.8542  0.8520  0.8526  0.8527  0.8530  0.8529  0.0007  
Fit time          141.26  141.60  142.31  147.27  144.34  143.36  2.23    
Test time         9.04    9.31    8.88    9.30    8.90    9.09    0.19    
Elapsed time: 788.20 seconds